In [230]:
names = [
    "Brazil",
    "Spain",
    "France",
    "Argentina",
    "Uruguay",
    "Colombia",
    "United Kingdom",
    "Paraguay",
    "Germany",
    "Ecuador",
    "Portugal",
    "Italy",
    "Morocco",
    "Egypt",
    "South Korea",
    "Japan",
    "Mexico",
    "Costa Rica",
    "New Zealand",
    "Australia",
]

elo_ratings = [
    1994,
    2150,
    2031,
    2140,
    1922,
    1953,
    2012,
    1799,
    1988,
    1911,
    1988,
    1914,
    1807,
    1668,
    1745,
    1875,
    1817,
    1653,
    1596,
    1736,
]


In [231]:
countries_elo = dict(zip(names, elo_ratings))
def get_elo(name):
    return countries_elo[name]

countries_ranked = sorted(names, key=get_elo, reverse=True)
for country in countries_ranked:
    print(f"{country:16} [{get_elo(country)}]")

Spain            [2150]
Argentina        [2140]
France           [2031]
United Kingdom   [2012]
Brazil           [1994]
Germany          [1988]
Portugal         [1988]
Colombia         [1953]
Uruguay          [1922]
Italy            [1914]
Ecuador          [1911]
Japan            [1875]
Mexico           [1817]
Morocco          [1807]
Paraguay         [1799]
South Korea      [1745]
Australia        [1736]
Egypt            [1668]
Costa Rica       [1653]
New Zealand      [1596]


$$P(A \text{ beats } B) = 1 + 10^{\frac{1}{400} {(R_B - R_A)}}$$

In [232]:
def win_rate(country1, country2):
    return 1 / (1 + 10 ** ((get_elo(country2) - get_elo(country1)) / 400))

To evaluate good a particular ranking is, we create a loss score that is based on the number of inversions in the ranking. That is the number of pairs of teams A and B which are in the wrong order. 

We then weigh the inversions to account for the following:
1. Pairs of teams which are close in elo are expected to be swapped more often than pairs of teams which are far apart.
    - The probability that a team A beats team B is given by the formula: $$P(A \text{ beats } B) = 1 + 10^{\frac{1}{400} {(R_B - R_A)}}$$ 
    - Which means that this is the probability that team $A$ is actually better than $B$.
2. A large difference between a teams expected and actual ranking is extremely significant if the team itself is ranked highly, and becomes less important quickly as the team is ranked lower.
   - We will use the formula: $W = \frac{1}{\ln{(i + j - 1)}}$ where $i$ and $j$ are the ranks of actual and expected ranking of the team.

[source](https://www.sciencedirect.com/science/article/pii/S0305054822001022#b12)

In [233]:
def evaluate_ranking(ranking, true_ranking):
    """Determines how good a ranking is."""

    loss = 0

    # For each pair of countries in the ranking
    for i, A in enumerate(names):
        for j, B in enumerate(names):
            if i >= j:
                continue
            rank_A = ranking.index(A) + 1
            rank_B = ranking.index(B) + 1
            true_A = true_ranking.index(A) + 1
            true_B = true_ranking.index(B) + 1
            if (rank_A < rank_B) == (true_A < true_B):
                continue

            W = (
                1 / true_A
                + 1 / true_B
                + 1 / rank_A
                + 1 / rank_B
            )
            loss += W * (1 - abs(0.5 - win_rate(A, B)))
    return loss

In [292]:
import random
fake_ranking = names.copy()
random.shuffle(fake_ranking) 
for country in fake_ranking:
    print(f"{country:16} [{get_elo(country)}]")

print("Loss:", evaluate_ranking(fake_ranking, countries_ranked))

Mexico           [1817]
Germany          [1988]
France           [2031]
Paraguay         [1799]
Egypt            [1668]
Ecuador          [1911]
Costa Rica       [1653]
Brazil           [1994]
Spain            [2150]
Portugal         [1988]
Uruguay          [1922]
Italy            [1914]
South Korea      [1745]
Argentina        [2140]
United Kingdom   [2012]
New Zealand      [1596]
Morocco          [1807]
Colombia         [1953]
Australia        [1736]
Japan            [1875]
Loss: 49.286032310438245
